In [1]:
import pandas as pd
import os
import numpy as np
folder_path = "Data"  # Make sure the folder path is correct

# Read each CSV file into a DataFrame
df_price = pd.read_csv(os.path.join(folder_path, "Prices.csv"))
df_chargesvar = pd.read_csv(os.path.join(folder_path, "Charges_var.csv"))
df_prod = pd.read_csv(os.path.join(folder_path, "Production.csv"))
df_plantation = pd.read_csv(os.path.join(folder_path, "plantation.csv"))
df_month_index = pd.read_csv(os.path.join(folder_path, "month_index.csv"))
df_sim=pd.read_csv(os.path.join(folder_path, "Simulation.csv"))
# Now you can work with these DataFrames as needed
df_price = df_price.fillna(0)
df_prod=df_prod.fillna(0)
serre_sau_dict = df_sim.set_index('serre')['SAU(ha)'].to_dict()


In [30]:
scenario_delai_dict = {}
scenario_duree_dict = {}
scenario_mois_dict = {}
scenario_culture={}
scenario_cmo={}
scenario_vitesse={}
scenario_cv={}
for index,row in df_chargesvar.iterrows():
    scenario = row['Scénario']
    cmo=row["Coûts de Main d'oeuvre"]
    vitesse=row["Vitesse de main d'œuvre kg/personne/jour"]
    cv=row["Coûts variable par hectar"]
    scenario_cmo[scenario]=cmo
    scenario_vitesse[scenario]=vitesse
    scenario_cv[scenario]=cv
for index, row in df_prod.iterrows():
    scenario = row['Scénario']
    delai = row['Délai pour début de production']
    duree = row['Durée de production en semaine']
    mois = row['Mois']
    culture=row['Culture']
    
    scenario_delai_dict[scenario] = delai
    scenario_duree_dict[scenario] = duree
    scenario_mois_dict[scenario] = mois
    scenario_culture[scenario] =culture
    
scenarios=list(scenario_culture.keys())


In [4]:
price={}
price["Framboise"]=np.array(df_price.iloc[0,1:])
price["Mure"]=np.array(df_price.iloc[1,1:])

In [15]:
secteur_serre_dict = {}

# Iterate over the rows of the DataFrame
for index, row in df_sim.iterrows():
    secteur = row['secteur']
    serre = row['serre']
    
    # If the secteur is not in the dictionary yet, create a new list
    # Otherwise, append the serre to the existing list
    if secteur not in secteur_serre_dict:
        secteur_serre_dict[int(secteur)] = [int(serre)]
    else:
        secteur_serre_dict[int(secteur)].append(int(serre))
serre_secteur_dict = {serre: secteur for secteur, serres in secteur_serre_dict.items() for serre in serres}
num_serre=df_sim.shape[0]
num_sect=len(secteur_serre_dict)
num_scenario=19

In [10]:
import numpy as np
prod_mat=np.matrix(df_prod.iloc[:,8:])

# Assuming num_serre, num_scenario, and prod_mat are defined somewhere
# Assuming price, scenario_culture, scenario_delai_dict, and scenario_duree_dict are also defined

# Initialize prod tensor
prod = np.zeros((num_serre, len(scenarios), 90))

def padded_dot(a, b):
    # Get the lengths of the arrays
    len_a = a.shape[1]
    len_b = b.shape[1]
    
    # Determine the maximum length
    max_len = max(len_a, len_b)
    
    # Pad arrays with zeros
    a_padded = np.pad(a, ((0, 0), (0, max_len - len_a)), mode='constant')
    b_padded = np.pad(b, ((0, 0), (0, max_len - len_b)), mode='constant')
    
    # Perform dot product
    result = np.dot(a_padded, b_padded.T)
    
    return result
    
    return result
for i in range(num_serre):
    for j in scenarios:
        for t in range(90):
            # Calculate dot product using numpy's dot function
            if t + scenario_delai_dict[j] + scenario_duree_dict[j]<=89:
                prod[(i, scenarios.index(j), t)] = np.dot(
    np.array(price[scenario_culture[j]][t + scenario_delai_dict[j]:
                                           t + scenario_delai_dict[j] 
                                           + scenario_duree_dict[j]]).reshape(1, -1),
    np.array(prod_mat[scenarios.index(j), 8:8 + scenario_duree_dict[j]]).reshape(-1, 1)
)
            elif np.array(prod_mat[scenarios.index(j), 8:8+89-t-scenario_delai_dict[j]]).shape[1]!=0:
                prod[(i, scenarios.index(j), t)] = padded_dot(
    np.array(price[scenario_culture[j]][t + scenario_delai_dict[j]:]).reshape(1, -1),
    np.array(prod_mat[scenarios.index(j), 8:8+89-t-scenario_delai_dict[j]]).reshape(1, -1)
)

array([[[183400.72955368, 179486.16977296, 183057.98150017, ...,
              0.        ,      0.        ,      0.        ],
        [ 78487.78384193,  84072.02376649,  92712.86909263, ...,
              0.        ,      0.        ,      0.        ],
        [198871.71613741, 182174.97666658, 177282.91323669, ...,
              0.        ,      0.        ,      0.        ],
        ...,
        [142885.33287429, 134060.42001011, 134256.18903487, ...,
              0.        ,      0.        ,      0.        ],
        [142885.33287429, 134060.42001011, 134256.18903487, ...,
              0.        ,      0.        ,      0.        ],
        [161925.21969701, 152249.81387409, 150490.51818565, ...,
              0.        ,      0.        ,      0.        ]],

       [[183400.72955368, 179486.16977296, 183057.98150017, ...,
              0.        ,      0.        ,      0.        ],
        [ 78487.78384193,  84072.02376649,  92712.86909263, ...,
              0.        ,      0.     

In [25]:
import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("portfolio")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")
    
    #Choice of Scenario par serre variables
    csc={}
    for i in range(num_serre):
        for j in scenarios:
            csc[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')
    #These are the constraints for the variables I defined Above
    #Meme scenario par secteur
    for i in range(num_sect):
        ref=secteur_serre_dict[i+1][0]
        for j in secteur_serre_dict[i+1]:
            for k in scenarios:
                m.addConstr(csc[(j-1,k)] ==csc[(ref,k)], f'c_0_{j}_{i}')
    #Cette contrainte est obtenue par la contraposé de la cinquième contrainte
    for i in range(num_sect):
        if i!=5:
            for j in secteur_serre_dict[i+1]:
                m.addConstr(csc[(j-1,4)]==0,f'c_1_{j}_{i}')
    #For now, we have done constraints 2 and 5, now let's do the rest
    
    #Variables de semaines
    #Choix d'unesemaine par serre
    choix_semaines={}
    for i in range(num_serre):
        for t in range(90):
            choix_semaines[(i,t)]=m.addVar(vtype=GRB.BINARY, name=f'cs_{i}_{t}')
        m.addConstr(gp.quicksum(choix_semaines[(i, t)] for t in range(90)) == 1, f"constraint_{i}")
        
    CA = m.addVar(vtype=GRB.CONTINUOUS, name="CA")
    for i in range(num_serre):
        for j in scenarios:
            for t in range(90):
                CA+=csc[(i, j)]*choix_semaines[(i, t)]*prod[(i, scenarios.index(j), t)]*serre_sau_dict[i+1]


    CMO= m.addVar(vtype=GRB.CONTINUOUS, name="CMO")
    for i in range(num_serre):
        for j in scenarios:
            for t in range(90):
                CMO+=csc[(i, j)]*choix_semaines[(i, t)]*\
                prod[(i, scenarios.index(j), t)]*serre_sau_dict[i+1]*\
                

                

    # Set objective
    m.setObjective(CA-CMO, GRB.MAXIMIZE)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 381 rows, 2500 columns and 2685 nonzeros
Model fingerprint: 0x36c3be32
Model has 22896 quadratic objective terms
Variable types: 1 continuous, 2499 integer (2499 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [4e+03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.000000e+09
Presolve removed 84 rows and 315 columns
Presolve time: 0.01s

Cleanup yields a better solution

Explored 0 nodes (0 simplex iterations) in 0.12 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 1e+30 2e+09 
No other solutions better than 0

Model is unbounded
Best objective 1.